##### Copyright 2026 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma 2: DeFi Protocol Development

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]DeFi_Protocol_Development.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>
<br clear="left"/>

Decentralized Finance (DeFi) has revolutionized financial services — enabling permissionless lending, trading, and yield generation on blockchain networks. Building DeFi protocols requires deep knowledge of Solidity, token standards, and economic mechanisms.

This notebook demonstrates how to use **Gemma 2 9B IT** as an AI-powered assistant for DeFi smart contract development. To fit the 9B model on a standard Colab T4 GPU, we use 4-bit quantization via `bitsandbytes`. You will learn how to generate smart contract code from natural language specifications.

> **Note:** AI-generated smart contracts are for educational purposes only. Always review, test, and audit code before any deployment.

**What you'll explore:**
1. **ERC-20 Token** — Generate a token using OpenZeppelin contracts
2. **AMM Concepts** — Understand automated market maker mechanics
3. **Staking Basics** — Learn reward distribution patterns
4. **Protocol Architecture** — Design how components work together

**Prerequisites:** Basic understanding of Solidity and Ethereum concepts.

**Author**: Rahul Lashkari  
**GitHub**: [github.com/rahul-lashkari](https://github.com/rahul-lashkari)

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Hugging Face Setup

To use the 9B model on a T4 GPU efficiently, we will use the Hugging Face hub.

1. **Create a Hugging Face Account:** If you don't already have one, sign up at [huggingface.co](https://huggingface.co/join).
2. **Accept Model Terms:** Visit the [Gemma 2 model page](https://huggingface.co/google/gemma-2-9b-it) and accept the usage conditions.
3. **Get Token:** Go to your [Settings > Access Tokens](https://huggingface.co/settings/tokens) and create a new token.

### Configure Hugging Face access token

Add your Hugging Face access token to Colab Secrets:

1. Open your Google Colab notebook and click on the 🔑 **Secrets** tab in the left panel.
2. Create a new secret with `HF_TOKEN` as the name and your [Hugging Face access token](https://huggingface.co/settings/tokens) as the value.
3. Toggle the button on the left to allow notebook access to the secret.

In [2]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

Install the required dependencies.

- Transformers: For loading and running the model
- Accelerate: For efficient model loading
- BitsAndBytes: For 4-bit quantization to fit larger models on T4 GPU

In [3]:
!pip install -q -U transformers accelerate bitsandbytes

## Load Gemma 2 (9B) with Quantization

We load the `google/gemma-2-9b-it` model. To fit it into the 16GB VRAM of a T4 GPU, we use **4-bit NF4 quantization**.

In [4]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

gc.collect()
torch.cuda.empty_cache()

model_id = "google/gemma-2-9b-it"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    token=os.environ["HF_TOKEN"]
)

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/464 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

### Clear GPU memory

In [5]:
torch.cuda.empty_cache()
print("Gemma 2 9B loaded successfully with 4-bit quantization!")

Gemma 2 9B loaded successfully with 4-bit quantization!


### Define Helper Function

We create a helper function to format prompts using the chat template and generate responses.

---

## Task 1: Generate an ERC-20 Token

[ERC-20](https://ethereum.org/en/developers/docs/standards/tokens/erc-20/) is the fungible token standard on Ethereum. Let's generate a governance token using OpenZeppelin's battle-tested contracts.

In [6]:
from IPython.display import Markdown, display

def generate_response(prompt, max_new_tokens=2048):
    """Generate a response from Gemma 2."""
    messages = [
        {"role": "user", "content": prompt}
    ]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True, return_dict=True).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

    # Decode only the generated response (skipping the prompt)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return response

def chat(prompt, max_new_tokens=2048):
    """Generate and display response as Markdown."""
    response = generate_response(prompt, max_new_tokens)
    display(Markdown(response))

# Generate ERC-20 token code
token_prompt = """Write a simple ERC-20 token contract in Solidity that:
- Is named "DeFi Governance Token" with symbol "DGT"
- Has 18 decimals and 100 million initial supply minted to deployer
- Inherits from OpenZeppelin v5.x: ERC20, ERC20Burnable, ERC20Pausable, Ownable
- Uses Solidity ^0.8.20

Keep it minimal - just inherit from OpenZeppelin contracts without custom logic."""

chat(token_prompt)

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.20;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Burnable.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Pausable.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

contract DeFiGovernanceToken is ERC20, ERC20Burnable, ERC20Pausable, Ownable {
    constructor() ERC20("DeFi Governance Token", "DGT") {
        _mint(msg.sender, 100000000 * 10**18);
    }
}
```

**Explanation:**

1. **Imports:** The contract imports necessary OpenZeppelin contracts for ERC20 functionality, burnable tokens, pausable tokens, and ownership control.
2. **Contract Definition:** The `DeFiGovernanceToken` contract inherits from all imported contracts, combining their functionalities.
3. **Constructor:**
   - The constructor initializes the ERC20 token with the name "DeFi Governance Token" and symbol "DGT".
   - It mints 100 million tokens (100,000,000 * 10^18) to the contract deployer (the address that deploys the contract).
4. **Functionality:**
   - The contract inherits the following functionalities from OpenZeppelin contracts:
     - **ERC20:** Standard ERC20 token functionality (transfer, approve, balance, etc.).
     - **ERC20Burnable:** Allows token holders to burn (destroy) their tokens.
     - **ERC20Pausable:** Allows the owner to pause and unpause token transfers.
     - **Ownable:** Provides ownership control, allowing the owner to perform certain actions.

**Key Points:**

- This contract provides a basic framework for a DeFi governance token.
- You can extend this contract with additional logic for specific governance features, such as voting, proposals, or delegation.
- Remember to thoroughly audit and test the contract before deploying it to a live network.





### Understanding the Generated Code

The generated ERC-20 token inherits from multiple OpenZeppelin contracts:

| Contract | Purpose |
|----------|---------|
| `ERC20` | Base token functionality (transfer, approve, etc.) |
| `ERC20Burnable` | Adds `burn()` and `burnFrom()` functions |
| `ERC20Pausable` | Pause mechanism that blocks all transfers |
| `Ownable` | Access control for admin functions |

This composition pattern is a DeFi best practice — using battle-tested, audited code instead of writing from scratch.

---
## Task 2: Explain AMM Mechanics

Before generating an AMM contract, let's have Gemma explain the core concepts.

In [7]:
amm_explain_prompt = """Explain Automated Market Makers (AMMs) for a developer:

1. How does the constant product formula (x * y = k) work?
2. Why do larger trades cause more slippage?
3. What is impermanent loss for liquidity providers?

Use a simple numerical example with a 1000/1000 token pool."""

chat(amm_explain_prompt)

## Automated Market Makers (AMMs) Explained for Developers

**1. The Constant Product Formula (x * y = k)**

AMMs like Uniswap use a constant product formula to determine the price of tokens within a liquidity pool. 

Let's say we have a pool with 1000 tokens of **Token A** and 1000 tokens of **Token B**. This means:

* **x = 1000** (amount of Token A)
* **y = 1000** (amount of Token B)
* **k = x * y = 1,000,000** (the constant product)

This formula ensures that the product of the amounts of both tokens in the pool always remains constant.

**How it works:**

* **Adding liquidity:** When a user adds liquidity to the pool, they provide both Token A and Token B in a ratio that maintains the constant product. For example, if a user adds 500 more of Token A, the pool will automatically adjust the amount of Token B to keep k = 1,000,000.
* **Swapping tokens:** When a user swaps one token for another, the AMM adjusts the amounts of both tokens in the pool to maintain the constant product.

**2. Slippage due to Larger Trades**

Slippage occurs when the price of a token changes during a trade, causing the user to receive a different amount than they expected.

**Larger trades cause more slippage because:**

* They significantly impact the ratio of tokens in the pool.
* This requires a larger adjustment to the pool's balance to maintain the constant product.
* The larger the adjustment, the more the price deviates from the initial price, leading to higher slippage.

**Example:**

Imagine a user wants to buy 500 Token A from our 1000/1000 pool.

* **Initial price:** 1 Token A = 1 Token B
* **Trade execution:** To buy 500 Token A, the pool needs to sell 500 Token B. This reduces the amount of Token B in the pool and increases the amount of Token A.
* **New price:** The price of Token A will likely increase slightly due to the imbalance, meaning the user receives slightly less Token A than they would have if the trade was smaller.

**3. Impermanent Loss for Liquidity Providers**

Impermanent loss occurs when the price of one token in a pool significantly changes relative to the other.

**Example:**

* **Initial state:** 1000 Token A / 1000 Token B
* **Price change:** Token A price increases significantly.

**Impact on liquidity providers:**

* **Token A holders:** They now have more Token A, but the value of their overall position may be lower because the price of Token B has decreased.
* **Token B holders:** They now have less Token B, but the value of their overall position may be higher because the price of Token A has increased.

**To illustrate:**

If Token A's price doubles, a liquidity provider who initially held 500 of each token would now have:

* 500 Token A (worth double)
* 500 Token B (worth half)

Their total value may be higher, but they would have been better off simply holding the tokens individually.

**Key takeaway:**

Impermanent loss is a risk for liquidity providers, as their returns may be lower than if they had simply held their tokens individually.




---
## Task 3: AMM Swap Function

Now let's generate the core swap function that implements the constant product formula.

In [8]:
amm_swap_prompt = """Write a Solidity function that calculates the output amount for an AMM swap using the constant product formula.

Given:
- reserveIn: current reserve of input token
- reserveOut: current reserve of output token
- amountIn: amount being swapped
- Fee: 0.3%

The function should:
1. Apply the 0.3% fee to amountIn
2. Calculate amountOut using the constant product formula with the fee-adjusted input
3. Return the output amount

Just write the pure calculation function, not a full contract."""

chat(amm_swap_prompt)

```solidity
function calculateSwapOutput(uint256 reserveIn, uint256 reserveOut, uint256 amountIn, uint256 fee) pure returns (uint256) {
    uint256 feeAmount = (amountIn * fee) / 10000;
    uint256 amountInAfterFee = amountIn - feeAmount;
    uint256 amountOut = (amountInAfterFee * reserveOut) / (reserveIn + amountInAfterFee);
    return amountOut;
}
``` 


**Explanation:**

1. **`feeAmount` Calculation:**
   -  `feeAmount = (amountIn * fee) / 10000;` calculates the fee amount as 0.3% of `amountIn`. We use 10000 to represent 100% for the percentage calculation.

2. **`amountInAfterFee` Calculation:**
   - `amountInAfterFee = amountIn - feeAmount;` subtracts the calculated `feeAmount` from the original `amountIn`.

3. **`amountOut` Calculation:**
   - `amountOut = (amountInAfterFee * reserveOut) / (reserveIn + amountInAfterFee);` applies the constant product formula to calculate the output amount.

4. **Return Value:**
   - The function returns the calculated `amountOut`.



Let me know if you have any other questions.


### Key AMM Concepts

**Constant Product Formula:**
```
reserveA * reserveB = k (constant)
```

**Swap Calculation (with 0.3% fee):**
```
amountInAfterFee = amountIn * (1 - 0.003)
amountOut = (amountInAfterFee * reserveOut) / (reserveIn + amountInAfterFee)
```

The fee accumulates value for liquidity providers over time.

---
## Task 4: Staking Reward Calculation

Staking contracts allow token holders to earn rewards. Let's understand the Synthetix reward distribution pattern.

In [9]:
staking_prompt = """Explain the Synthetix StakingRewards pattern for distributing rewards:

1. What is `rewardPerTokenStored` and how is it updated?
2. How does `earned(user)` calculate rewards without looping?
3. Why is this pattern gas-efficient?

Include the core formulas used in the calculations."""

chat(staking_prompt)

## Synthetix StakingRewards: A Gas-Efficient Reward Distribution Pattern

The Synthetix StakingRewards contract employs a clever pattern for distributing rewards efficiently, minimizing gas consumption. Let's break down the key components:

**1. `rewardPerTokenStored`:**

This variable represents the **cumulative reward per token** that has been earned but not yet distributed. It's a crucial state variable that tracks the total rewards accrued for all staked tokens.

**How it's updated:**

- **`_updateRewardPerToken()`:** This function is called whenever new rewards are added to the contract. It calculates the new `rewardPerTokenStored` based on the total rewards added (`_reward`) and the total supply of staked tokens (`_totalSupply`).

**Formula:**

```
rewardPerTokenStored = (rewardPerTokenStored * _totalSupply + _reward) / _totalSupply
```

**2. `earned(user)`:**

This function calculates the **amount of rewards earned by a specific user** without looping through all staked tokens. It leverages the `rewardPerTokenStored` and the user's staked balance (`userBalance`) to achieve this efficiently.

**Formula:**

```
earned(user) = userBalance * (rewardPerTokenStored - user.rewardPerToken)
```

**Explanation:**

- `userBalance`: The number of tokens staked by the user.
- `user.rewardPerToken`: The last recorded reward per token for the user. This value is updated when the user claims their rewards.
- The difference between `rewardPerTokenStored` and `user.rewardPerToken` represents the **new rewards earned by the user since their last claim**.

**3. Gas Efficiency:**

This pattern is gas-efficient because:

- **No looping:** `earned(user)` avoids iterating through all staked tokens, significantly reducing gas consumption.
- **State variable optimization:** `rewardPerTokenStored` efficiently tracks the cumulative rewards, eliminating the need for complex calculations on each individual user's stake.

**Overall, the Synthetix StakingRewards pattern demonstrates a well-designed approach to reward distribution, balancing user experience with gas efficiency.**


### Synthetix Reward Pattern Summary

The pattern uses an accumulator to track rewards efficiently:

```
rewardPerToken = rewardPerToken + (reward * 1e18 / totalStaked)
earned = balance * (rewardPerToken - userPaid) / 1e18
```

This achieves O(1) complexity - no loops needed regardless of staker count.

---
## Task 5: Protocol Architecture

Let's ask Gemma how these DeFi primitives can work together.

In [10]:
architecture_prompt = """Describe how these three DeFi components work together:

1. ERC-20 governance token (DGT)
2. AMM liquidity pool (DGT/ETH pair)
3. Staking contract (stake DGT, earn DGT)

Briefly explain:
- How initial liquidity could be bootstrapped
- How staking rewards could be funded
- The deployment order for these contracts"""

chat(architecture_prompt)

Here's how these DeFi components work together:

**1. ERC-20 Governance Token (DGT)**

* **Purpose:** DGT is the native token of the DeFi protocol. It grants holders voting rights on protocol upgrades, fee structures, and other important decisions.

**2. AMM Liquidity Pool (DGT/ETH Pair)**

* **Purpose:** This automated market maker (AMM) allows users to swap DGT for ETH and vice versa. The pool is filled with DGT and ETH tokens, and the price is determined algorithmically based on the ratio of tokens in the pool.

**3. Staking Contract (Stake DGT, Earn DGT)**

* **Purpose:** This contract allows users to lock up their DGT tokens and earn rewards in the form of additional DGT. This incentivizes users to hold DGT and participate in the protocol's governance.

**How They Work Together**

* **Trading:** Users can trade DGT for ETH and vice versa using the AMM liquidity pool. This creates market liquidity and allows for price discovery.
* **Staking:** Users can stake their DGT tokens in the staking contract to earn rewards. These rewards are funded by a portion of the trading fees generated by the AMM.
* **Governance:** DGT holders can use their tokens to vote on proposals that affect the protocol. This ensures that the protocol is governed by the community.

**Bootstrapping Initial Liquidity**

* **Initial DEX Offering (IDO):** The project could launch an IDO, selling DGT tokens to early investors in exchange for ETH. These ETH funds could then be used to buy DGT and bootstrap the liquidity pool.
* **Team/Advisor Allocation:** A portion of the DGT tokens could be allocated to the team and advisors, who could then contribute liquidity to the pool.
* **Liquidity Mining:** The project could offer a bonus to early liquidity providers, incentivizing them to contribute to the pool.

**Funding Staking Rewards**

* **Trading Fees:** A portion of the trading fees generated by the AMM could be allocated to the staking contract as rewards.
* **Protocol Revenue:** If the protocol generates revenue from other sources (e.g., subscriptions, premium features), a portion of this revenue could be used to fund staking rewards.
* **Treasury:** The project could allocate a portion of its treasury funds to pay staking rewards.

**Deployment Order**

1. **ERC-20 Governance Token (DGT):** This contract needs to be deployed first to define the token's functionality and distribution.
2. **Staking Contract:** This contract can be deployed after the DGT token is live, allowing users to stake their tokens and earn rewards.
3. **AMM Liquidity Pool (DGT/ETH Pair):** This contract can be deployed after the DGT token and staking contract are live, providing a platform for trading and liquidity.



Let me know if you have any other questions.


---
## Conclusion

In this notebook, you used **Gemma 2 9B IT** to explore DeFi protocol development:

| Component | What You Learned |
|-----------|------------------|
| **ERC-20 Token** | Generate tokens using OpenZeppelin inheritance |
| **AMM Mechanics** | Constant product formula and swap calculations |
| **Staking Pattern** | Synthetix-style reward distribution |
| **Protocol Design** | How DeFi primitives connect together |

**Key Takeaways:**
- Gemma can explain DeFi concepts and generate Solidity code
- Always review and test AI-generated contracts before deployment
- Use established patterns (OpenZeppelin, Synthetix) for security
- Start with simple building blocks before creating complex protocols

**Next Steps:**
- Deploy sample contracts to a testnet (Sepolia, Holesky)
- Write tests with Foundry or Hardhat
- Get a professional security audit before any mainnet deployment

---
*For smart contract security analysis, see the companion notebook: [Smart Contract Auditing](https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Smart_Contract_Auditing.ipynb)*